In [1]:
import numpy as np
import pandas as pd

In [35]:
df = pd.read_csv('../dataset/wk9/outcome_data.csv')

target = 'Responded'

In [36]:
df[df["Outcome"] == target]

,District,House Type,Income,Previous Customer,Outcome
2,Rural,Detached,High,No,Responded
3,Urban,Semi-detached,High,No,Responded
4,Urban,Semi-detached,Low,No,Responded
6,Rural,Semi-detached,Low,Yes,Responded
8,Suburban,Semi-detached,Low,No,Responded
9,Urban,Terrace,Low,No,Responded
10,Suburban,Terrace,Low,Yes,Responded
11,Rural,Terrace,High,Yes,Responded
12,Rural,Detached,Low,No,Responded


In [37]:
np.where(df["Outcome"] == target, 1, 0)

array([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0])

In [38]:
df.describe()

,District,House Type,Income,Previous Customer,Outcome
count,14,14,14,14,14
unique,3,3,2,2,2
top,Suburban,Semi-detached,High,No,Responded
freq,5,5,7,8,9
